In [1]:
# Copyright 2015 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

from __future__ import absolute_import
from __future__ import print_function

import collections
import math
import os
import random
import zipfile
import pickle

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
import shelve
import re

from scipy import spatial

In [2]:
# Step 1: Download the data.
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urllib.request.urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified', filename)
    else:
        print(statinfo.st_size)
        raise Exception(
            'Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
# Read the data into a list of strings.
def read_data(filename):
    """Extract the first file enclosed in a zip file as a list of words"""
    with zipfile.ZipFile(filename) as f:
        data = f.read(f.namelist()[0]).split()
    return data

words = read_data(filename)
print('Data size', len(words))

Data size 17005207


In [4]:
sensor_dict = shelve.open('metadata/bacnet_devices.shelve','r')
buildingName = 'ebu3b'
naeDict = dict()
naeDict['bonner'] = ["607", "608", "609", "557", "610"]
naeDict['ap_m'] = ['514', '513','604']
naeDict['bsb'] = ['519', '568', '567', '566', '564', '565']
naeDict['ebu3b'] = ["505", "506"]
naeList = naeDict[buildingName]
wordList = list()

tokenType = 'Alphanumeric'
tokenTypeList = ['NoNumber', 'Alphanumeric', 'AlphaAndNum', 'NumAsSingleWord']

for nae in naeList:
    device = sensor_dict[nae]
    h_dev = device['props']
    for sensor in device['objs']:
        h_obj = sensor['props']
        source_id = str(h_dev['device_id']) + '_' + str(h_obj['type']) + '_' + str(h_obj['instance'])
        #if h_obj['type'] not in (0,1,2,3,4,5,13,14,19):
        #    continue
        
        #create individual lists
        #jciList = ['jciname_'+val for val in re.findall("[a-zA-Z]+|\d+", sensor['jci_name'])]
        #nameList = ['name_'+val for val in re.findall("[a-zA-Z]+|\d+", sensor['name'])]
        #descList = ['desc_'+val for val in re.findall("[a-zA-Z]+|\d+", sensor['desc'])]
        if tokenType=='Alphanumeric':
            jciList = re.findall("\w+", sensor['jci_name'])
            nameList = re.findall("\w+", sensor['name'])
            descList = re.findall("\w+", sensor['desc'])
        elif tokenType in ['AlphaAndNum', 'NumAsSingleWord']:
            jciList = re.findall("[a-zA-Z]+|\d+", sensor['jci_name'])
            nameList = re.findall("[a-zA-Z]+|\d+", sensor['name'])
            descList = re.findall("[a-zA-Z]+|\d+", sensor['desc'])
        elif tokenType=='NoNumber':
            jciList = re.findall("[a-zA-Z]+|\d+", sensor['jci_name'])
            nameList = re.findall("[a-zA-Z]+|\d+", sensor['name'])
            descList =re.findall("[a-zA-Z]+|\d+", sensor['desc'])        
        else:
            assert(False)
        
        localList =  jciList + ['DOT'] + nameList + ['DOT'] + descList + ['\n']  
        if tokenType=='NumAsSingleWord':
            localList = ['NUM' if len(re.findall('\d+',word))>0 else word for word in localList]
        wordList = wordList + localList
        
words = wordList

In [5]:
b = collections.Counter(words)

In [6]:
# Step 2: Build the dictionary and replace rare words with UNK token.
vocabulary_size = len(b.keys())
print(vocabulary_size)

def build_dataset(words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
del words  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10])

data_index = 0

1107
Most common words (+UNK) [['UNK', 1], ('DOT', 10040), ('\n', 5020), ('NAE', 5006), ('N2', 4842)]
Sample data [3, 10, 5, 1, 3, 10, 1, 983, 920, 989]


In [7]:
print('EBU3B' in count)

False


In [8]:
# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1 # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [ skip_window ]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
    print(batch[i], '->', labels[i, 0])
    print(reverse_dictionary[batch[i]], '->', reverse_dictionary[labels[i, 0]])

10 -> 3
05 -> NAE
10 -> 5
05 -> EBU3B
5 -> 10
EBU3B -> 05
5 -> 1
EBU3B -> DOT
1 -> 5
DOT -> EBU3B
1 -> 3
DOT -> NAE
3 -> 1
NAE -> DOT
3 -> 10
NAE -> 05


In [ ]:
# Step 4: Build and train a skip-gram model.

batch_size = 128
#embedding_size = 128  # Dimension of the embedding vector.
embedding_size = 64  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default():

    # Input data. q
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
        embeddings = tf.Variable(
            tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
        nce_weights = tf.Variable(
            tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

    # Compute the average NCE loss for the batch.
    # tf.nce_loss automatically draws a new sample of the negative labels each
    # time we evaluate the loss.
    loss = tf.reduce_mean(
        tf.nn.nce_loss(nce_weights, nce_biases, embed, train_labels,
                     num_sampled, vocabulary_size))

    # Construct the SGD optimizer using a learning rate of 1.0.
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(
        normalized_embeddings, valid_dataset)
    similarity = tf.matmul(
        valid_embeddings, normalized_embeddings, transpose_b=True)

In [ ]:
# Step 5: Begin training.
num_steps = 100001

with tf.Session(graph=graph) as session:
    # We must initialize all variables before we use them.
    tf.initialize_all_variables().run()
    print("Initialized")

    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = generate_batch(
            batch_size, num_skips, skip_window)
        feed_dict = {train_inputs : batch_inputs, train_labels : batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print("Average loss at step ", step, ": ", average_loss)
            average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in xrange(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8 # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k+1]
                log_str = "Nearest to %s:" % valid_word
                for k in xrange(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = "%s %s," % (log_str, close_word)
                print(log_str)
    final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step  0 :  156.658828735
Nearest to CO2: 2138, VMA107, TEMP, Sen, Server, Cool, Activated, VMA172,
Nearest to HTGPID: EXCHANGER, 3213, Floor, 21, FAULT, IN, OCC, rm,
Nearest to FAN: 146, 3126, HW, HEATING, PM1TI, DHWS, Temps, 3220,
Nearest to 161: 4128, VMA160, VMA102, Time, VMA151, VMA167, VMA157, 4207,
Nearest to DMPR: HX1B, Sunbeams, 1150, EF4, 113, inUnocc, 1, HUBBS_WIND,
Nearest to Occ: HUBBS_ENTHALPY, FSCHW, south, Tuning, B250, 4203, Trend2, OCCUPIED,
Nearest to Shed: 127, Stages, Liquid, DO1, Air, 3106, VMA181, VMA108,
Nearest to Occupied: Exchanger, HX, Average, inUnocc, Schedule, FREQ, EBU2_ZENITH, Cooling,
Nearest to ADJ: 15, 1200C, STANDBY, 2118, SS, Loss, Maintenance, 4262,
Nearest to Adjust: 160, 1126, 3219N, DC, VMA124, 3226, DO1, Zone,
Nearest to Temperature: VND23, 142, HI, OCCSENS, Hubbs, ACC, VMA112, Exhaust,
Nearest to DX: 1155, 4202, STATIC, Box, 1st, 4262, Basement, VMA177,
Nearest to Cooling: OVRD, setpoint, SF1, CUR, DA1, 4138, Econ, 

In [ ]:
# Step 6: Visualize the embeddings.

def plot_with_labels(low_dim_embs, labels, filename='figs/embedding_'+buildingName+'_' + tokenType + '.png'):
    assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
    plt.figure(figsize=(18, 18))  #in inches
    for i, label in enumerate(labels):
        if label.lower()=='num':
            c= 'red'
            print("RED!!!!")
        else:
            c = 'black'
        x, y = low_dim_embs[i,:]
        plt.scatter(x, y)
        plt.annotate(label,
                 xy=(x, y),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom',
                    color=c)

    plt.savefig(filename)

try:
    #import matplotlib
    #reload(matplotlib)
    #matplotlib.use('Agg')
    from sklearn.manifold import TSNE
    import matplotlib.pyplot as plt
    %matplotlib inline
    

    tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
    plot_only = vocabulary_size
    low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only,:])
    labels = [reverse_dictionary[i] for i in xrange(plot_only)]
    plot_with_labels(low_dim_embs, labels)
    with open('data/embedding_'+buildingName+'_' + tokenType + '.pkl', 'wb') as fp:
        pickle.dump([final_embeddings, dictionary, reverse_dictionary], fp)

except ImportError:
    print("Please install sklearn and matplotlib to visualize embeddings.")